In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [ ]:
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# validation_images, validation_labels = train_images[:5000], train_labels[:5000]
train_images, train_labels = train_images[35000:], train_labels[35000:]

test_images, test_labels = test_images[7000:], test_labels[7000:]

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
# train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
# print("Training data size:", train_ds_size)
# validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [ ]:
# from sklearn.model_selection import train_test_split

# # (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# # print('x_train shape:', x_train.shape)
# # print('X shape: ', x_test.shape)
# # print('Y shape: ', x_test.shape)

# second_size = 25000/50000

# X1, X2 = train_test_split(train_ds, test_size=second_size)
# Xt1, Xt2 = train_test_split(test_ds, test_size=second_size)

# train_ds = X2
# test_ds = Xt2

# print('X_train shape: ', x_train.shape)
# print('X_test shape: ', x_test.shape)
# print('Y_train shape: ', y_train.shape)
# print('Y_test shape: ', y_test.shape)

In [ ]:
plt.figure(figsize=(20,20))
for i, (image, label) in enumerate(train_ds.take(5)):
    ax = plt.subplot(5,5,i+1)
    plt.imshow(image)
    plt.title(CLASS_NAMES[label.numpy()[0]])
    plt.axis('off')

In [ ]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
# validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
# print("Validation data size:", validation_ds_size)

In [ ]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
# validation_ds = (validation_ds
#                   .map(process_images)
#                   .shuffle(buffer_size=train_ds_size)
#                   .batch(batch_size=32, drop_remainder=True))

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    # keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    # keras.layers.BatchNormalization(),
    # keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    # keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    # keras.layers.BatchNormalization(),
    # keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    # keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=96, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    # keras.layers.Dense(4096, activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_ds,
          epochs=5,
          # validation_data=validation_ds,
          # validation_freq=1,
          callbacks=[tensorboard_cb])

In [ ]:
model.evaluate(test_ds)

In [ ]:
x = 5
x